# Ampliación de datos

En este notebook se va a aplicar la técnica de ampliación de datos a un conjunto de reseñas de Google Maps separadas en dos ficheros: uno con las reseñas que se van a considerar válidas y el otro con las inválidas. Cada línea es una reseña nueva.

### Imports

In [1]:
import pandas as pd
from deep_translator import (GoogleTranslator, MyMemoryTranslator)
import copy
import time

### Direcorio de datos

In [2]:
validReviewsPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidReviewsPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

### Pandas
Se van a pasar los datos a dataframes: uno con las valoraciones validas y otro con las negativas. Cada fila del dataframe será una reseña

In [3]:
def importFromTxtToDF(source):
    with open(source, 'r', encoding="utf-8") as file:
        #Generate a list with all the reviews
        targetList = [line.strip() for line in file]

    targetDF = pd.DataFrame(targetList, columns=['Text'])
    return targetDF

In [4]:
#Read the file with the valid reviews
validReviewsDF = importFromTxtToDF(validReviewsPath)
#Read the file with the invalid reviews
invalidReviewsDF = importFromTxtToDF(invalidReviewsPath)

Se muestran las primeras reseñas válidas

In [5]:
validReviewsDF.head()

,Text
0,"""Tiene fácil acceso para las personas con movi..."
1,"""Espero que hayan mejorais"""
2,"""La estación es antigua, aparte de tener una s..."
3,"""Bien"""
4,"""Bonito comodo"""


Se muestran las primeras reseñas inválidas

In [6]:
invalidReviewsDF.head()

,Text
0,"""He vivido 35 años en el barrio y reconozco qu..."
1,"""localización con muchos bares interesantes"""
2,"""…"""
3,"""Muy rica comida.."""
4,"""Estación del.metro"""


## Medidas de similitud
Para poder comparar frases y seleccionar las mejores para generar los mejores datasets se van a desarrollar las siguientes medidas de similitud.

### Similitud Semántica

A continuación se va a diseñar una función para calcular la similitud semántica entre pares de oraciones. Es decir, se van a calcular los embeddings de oraciones de cada par de frases y se va a usar una métrica de similitud para ver como de parecido es el significado de ambas frases.

Se va a usar una versión de SBERT, llamada MiniLM (Minimal Lenguaje Model), que utiliza una variante más pequeña. Se usa MiniLM de seis capas (L6), que logra una precisón buena con menos recursos.

Este modelo fue entrenado usando un dataset que incluye datos en varios idiomas, entre ellos el español. Consecuentemente, no hay problema al introducir frases en castellano. Es cierto, que obtiene mejores resultados para frases en inglés, ya que se entreno con más datos en este idioma.

MiniLM es un modelo específicamente entrenado para mapear frases y parrafos a un espacio vectorial de 384 dimensiones. Es decir, este modelo permite obtener un embedding de una frase directamente. Usando otros modelos esta tarea no es posible de forma directa, ya que devuelven un embedding para cada palabra del texto.

El método de similitud que se va a usar es la similitud del coseno, por lo que los valores más cercanos a uno indicarán una mayor similitud entre las frases

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

#Given two texts and a model, the semantic similarity of the texts is returned
def getSemanticSimilarity(text1, text2, model):
    #Get the embeddings of the senteces
    embedding1 = model.encode(text1)
    embedding2 = model.encode(text2)

    #Get the cosine similarity of the senteces
    similarity = cosine_similarity([embedding1], [embedding2])

    return similarity[0][0]

### Similitud Léxica
Se va a diseñar una función para calcular la similitud léxica entre pares de oraciones. La similitud léxica mide el grado de coincidencia de palabras o términos entre dos frases o textos, sin tener en cuenta el significado subyacente.

Hay varias formas de realizar este cálculo: similitud del coseno basada en frecuencia de palabras, coeficiente de Jaccard,coeficiente de Dice ...

En este caso, se cree que la mejor opción es usar el coeficiente de Jaccard ya que calcula la similitud en función de la proporción de palabras comunes sobre el total de palabras únicas. Consecuentemente, esto nos permitirá detectar frases con menos coincidencias exactas en palabras.

Cuanto más cercano a uno sea el coeficiente de Jaccard más similares léxicamente serán las frases.

In [3]:
import unicodedata
import re

#Clean up the text removing punctuation, accent marks and convertin everything to lowercase
def cleanText(text):
    text = unicodedata.normalize('NFKD', text.lower()).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [8]:
def revomeSpanishStopWords(text):
    determinantes = {"el", "la", "los", "las", "un", "una", "unos", "unas", "este", "esta", "estos", "estas",
                 "ese", "esa", "esos", "esas", "aquel", "aquella", "aquellos", "aquellas", "mi", "mis",
                 "tu", "tus", "su", "sus", "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", 
                 "vuestra", "vuestros", "vuestras", "primer", "primero", "primera", "segundo", "segunda"}

    preposiciones = {"a", "ante", "bajo", "cabe", "con", "contra", "de", "desde", "durante", "en", "entre", 
                 "hacia", "hasta", "mediante", "para", "por", "según", "sin", "sobre", "tras", "versus", "vía"}

    conjunciones = {"y", "e", "ni", "o", "u", "pero", "sino", "sino que", "mas", "aunque", "que", "porque", 
                "como", "cuando", "donde", "mientras", "para que", "a fin de que", "puesto que", "ya que", 
                "si", "siempre que"}
    pronombres = {
        # Pronombres personales
        "yo", "tú", "vos", "él", "ella", "nosotros", "nosotras", 
        "vosotros", "vosotras", "ellos", "ellas", "usted", "ustedes",
        "me", "te", "lo", "la", "nos", "os", "los", "las", "le", "les", "se",
    
        # Pronombres posesivos
        "mío", "mía", "míos", "mías", 
        "tuyo", "tuya", "tuyos", "tuyas", 
        "suyo", "suya", "suyos", "suyas", 
        "nuestro", "nuestra", "nuestros", "nuestras", 
        "vuestro", "vuestra", "vuestros", "vuestras",
    
        # Pronombres demostrativos
        "este", "esta", "estos", "estas", 
        "ese", "esa", "esos", "esas", 
        "aquel", "aquella", "aquellos", "aquellas",
    
        # Pronombres relativos
        "que", "cual", "cuales", "quien", "quienes", 
        "cuyo", "cuya", "cuyos", "cuyas", "donde",
    
        # Pronombres interrogativos y exclamativos
        "qué", "quién", "quiénes", "cuál", "cuáles", 
        "cuánto", "cuánta", "cuántos", "cuántas", 
        "dónde", "cómo", "cuándo",
    
        # Pronombres indefinidos
        "alguien", "algo", "nadie", "nada", "cualquiera", 
        "todos", "todas", "varios", "varias", "muchos", 
        "muchas", "pocos", "pocas", "alguno", "alguna", 
        "algunos", "algunas", "ninguno", "ninguna", 
        "uno", "una", "unos", "unas", "demás"
    }

    #Combine all the words in one set
    spanishStopWords = determinantes | preposiciones | conjunciones | pronombres

    textWithoutStopWords = [word for word in text.split() if word.lower() not in spanishStopWords]

    return " ".join(textWithoutStopWords)

In [4]:
#Jaccard similarity
def jaccardSimilarity(text1, text2):
    #Get the set of words of each text
    wordsInText1 = set(revomeSpanishStopWords(cleanText(text1)).split())
    wordsInText2 = set(revomeSpanishStopWords(cleanText(text2)).split())

    intersection = len(wordsInText1.intersection(wordsInText2)) 
    union = len(wordsInText1.union(wordsInText2))
    
    if union == 0:
        return 0

    #intersection / union
    return intersection / union

In [5]:
#Given two texts, the Jaccard similarity of those texts is returned
def getLexicalSimilarity(text1, text2):
    return jaccardSimilarity(text1, text2)

## Retrotraducción

El primer método de ampliación de datos que se va a usar va a ser la retrotraducción. Consiste en traducir el texto a un idioma distinto y luego volverlo a traducir al idioma original. 

Este proceso puede genera texto con el mismo significado que el original pero distintas palabras.

In [7]:
def BackTranslation(translatorsList, reviewsDF, targetPath):
    #Generate a data list to store the text that has to be translated
    notTranslatedList = reviewsDF['Text'].tolist()

    #Translate the text as many times as needed
    for translator in translatorsList:
        #Generate a data frame to store the text that has been translated
        translatedList = []
        for elem in notTranslatedList:
            #Translate all the reviews
            try:
                translation = translator.translate(elem)
            except Exception as e: #If the translation fails "" is written
                translation = '""'
            #If an error ocurred translate it to a ""
            if translation == None:
                translation = '""'
                
            #Save the translations in the corresponding list 
            translatedList.append(translation)

            #Wait 0.2 seconds not to collapse the server
            time.sleep(0.2)

        #Prepare to translate again if needed
        notTranslatedList = copy.deepcopy(translatedList) 
       
    #Open the file in which the translations are strored
    translationFile = open(targetPath, 'w', encoding="utf-8")
    #write all the translations
    for elem in translatedList:
        translationFile.write(elem + "\n")
    #Close the file
    translationFile.close()
    
    return pd.DataFrame(translatedList, columns=['Text'])

### Google Translator

Primero se va a traducir del castellano al ingles y luego del inglés al castellano

In [8]:
validPath = '1. Back Translation\\1. Google Translator\\ValidReviewsTranslationsEsEnEnEs.txt'
invalidPath = '1. Back Translation\\1. Google Translator\\InvalidReviewsTranslationsEsEnEnEs.txt'

firstTranslator = GoogleTranslator(source = 'es', target = 'en')
secondTranslator = GoogleTranslator(source='en', target='es')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsGoogleEsEnEnEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsGoogleEsEnEnESDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

A continuación se va a traducir del castellano al japonés y del japonés al castellano

In [9]:
validPath = '1. Back Translation\\1. Google Translator\\ValidReviewsTranslationsEsJaJaEs.txt'
invalidPath = '1. Back Translation\\1. Google Translator\\InvalidReviewsTranslationsEsJaJaEs.txt'

firstTranslator = GoogleTranslator(source = 'es', target = 'ja')
secondTranslator = GoogleTranslator(source='ja', target='es')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsGoogleEsJaJaEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsGoogleEsJaJaEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

Por último se va a implementar una cadena de traducciones más larga: castellano a frances, frances a japones, japones a ruso y ruso a catellano.

In [10]:
validPath = '1. Back Translation\\1. Google Translator\\ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'
invalidPath = '1. Back Translation\\1. Google Translator\\InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'

firstTranslator = GoogleTranslator(source = 'es', target = 'fr')
secondTranslator = GoogleTranslator(source='fr', target='ja')
thirdTranslator = GoogleTranslator(source='ja', target='ru')
fourthTranslator = GoogleTranslator(source='ru', target='es')

translatorList = [firstTranslator, secondTranslator, thirdTranslator, fourthTranslator]

validSpanishReviewsGoogleEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsGoogleEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

### MyMemory Translator

Se van a realizar las mismas traducciones pero usando otro traductor

Castellano -> Inglés -> Castellano

In [11]:
validPath = '1. Back Translation\\2. MyMemory Translator\\ValidReviewsTranslationsEsEnEnEs.txt'
invalidPath = '1. Back Translation\\2. MyMemory Translator\\InvalidReviewsTranslationsEsEnEnEs.txt'

firstTranslator = MyMemoryTranslator(source = 'spanish', target = 'english')
secondTranslator = MyMemoryTranslator(source='english', target='spanish')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsMyMemoryEsEnEnEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsMyMemoryEsEnEnEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

Castellano -> Japonés -> Castellano

In [12]:
validPath = '1. Back Translation\\2. MyMemory Translator\\ValidReviewsTranslationsEsJaJaEs.txt'
invalidPath = '1. Back Translation\\2. MyMemory Translator\\InvalidReviewsTranslationsEsJaJaEs.txt'

firstTranslator = MyMemoryTranslator(source = 'spanish', target = 'japanese')
secondTranslator = MyMemoryTranslator(source='japanese', target='spanish')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsMyMemoryEsJaJaEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsMyMemoryEsJaJaEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

Castellano -> Francés -> Japonés -> Ruso -> Castellano

In [13]:
validPath = '1. Back Translation\\2. MyMemory Translator\\ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'
invalidPath = '1. Back Translation\\2. MyMemory Translator\\InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'

firstTranslator = MyMemoryTranslator(source = 'spanish', target = 'french')
secondTranslator = MyMemoryTranslator(source='french', target='japanese')
thirdTranslator = MyMemoryTranslator(source='japanese', target='russian')
fourthTranslator = MyMemoryTranslator(source='russian', target='spanish')

translatorList = [firstTranslator, secondTranslator, thirdTranslator, fourthTranslator]

validSpanishReviewsMyMemoryEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsMyMemoryEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

## Análisis de la retrotraducción y selección de los datos:

Dado que el código correspondiente a la traducción llevó largo rato y se dejo a la noche ejecutando, se vuelven a importar los datos a dataframes:

#### MyMemory genera errores en la traducción debido a problemas de conexión con el servidor. Consecuentemente, se procede a anilizar únicamente los datos generados por el traductor de Google

In [6]:
def importFromTxtToList(source):
    with open(source, 'r', encoding="utf-8") as file:
        #Generate a list with all the reviews
        targetList = [line.strip() for line in file]
    return targetList

Frases originales

In [7]:
validPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validPath)
invalidOriginal = importFromTxtToList(invalidPath) 

Se importan las frases traducidas de: Castellano -> Inglés -> Castellano

In [8]:
validPath = '1. Back Translation/1. Google Translator/ValidReviewsTranslationsEsEnEnEs.txt'
invalidPath = '1. Back Translation/1. Google Translator/InvalidReviewsTranslationsEsEnEnEs.txt'

validEsEnEnEsTraductionList = importFromTxtToList(validPath)
invalidEsEnEnEsTraductionList = importFromTxtToList(invalidPath) 

Castellano -> Japonés -> Castellano

In [9]:
validPath = '1. Back Translation/1. Google Translator/ValidReviewsTranslationsEsJaJaEs.txt'
invalidPath = '1. Back Translation/1. Google Translator/InvalidReviewsTranslationsEsJaJaEs.txt'

validEsJaJaEsTraductionList = importFromTxtToList(validPath)
invalidEsJaJaEsTraductionList = importFromTxtToList(invalidPath)

Castellano -> Francés -> Japonés -> Ruso -> Castellano

In [10]:
validPath = '1. Back Translation/1. Google Translator/ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'
invalidPath = '1. Back Translation/1. Google Translator/InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'

validEsFrFrJaJaRuRuEsList = importFromTxtToList(validPath)
invalidEsFrFrJaJaRuRuEsList = importFromTxtToList(invalidPath)

Para cada frase del conjunto de datos original (las frases con las reseñas válidas e inválidas), se va a calcular la similitud semántica (usando a un modelo basado en SBERT, conocido como MiniLM, que es más ligero y rápido consiguiendo resultados bastante certeros) y léxica con sus tres correspondientes frases generadas mediante el métodod de retrotraducción. Se van a seleccionar las frases que tengan mayor similitud semántica y menor similitud léxica y se van a guardar en un fichero para su posterior uso.

Se van a generar dos ficheros: un csv con dos elementos por fila (la frase original y la retrotraducción escogida) y el otro con solo las retrotraducciones seleccionadas.

In [12]:
from sentence_transformers import SentenceTransformer

#originalDataList: list of texts representing the original dataset
#allAugmentedDataList: list of list of texts representing the aumented data 
#(allAugmentedDataList = [augmentedDataList1, ... ,augmentedDataListN], where augmentedDataList = [augmentedData1, ..., augmentedDataM])
#pathWithOriginal: path of the csv with two columns (the original text and the best augmented text)
#pathAugmentedData: path of the file with only the augmented data (without the original text)
def processAugmentation(originalDataList, allAugmentedDataList, pathWithOriginal, pathAugmentedData):
    #Select the model for the semantic similarity
    model = SentenceTransformer('all-MiniLM-L6-v2')

    #Open the files in which the augmented data will be strored
    withOriginalFile = open(pathWithOriginal, "w", encoding="utf-8")
    augmentedDataFile = open(pathAugmentedData, "w", encoding="utf-8")

    #Write the titles of the csv
    withOriginalFile.write("OriginalText,AugmentedText,SemanticSimilarity,LexicalSimilarity\n")
    
    resul = []
    #Analize every phrase in the original data
    for i, originalText in enumerate(originalDataList):
        allAugmentedDataInfoDict = {}
        bestIdx = 1
        
        #Analize every traduction
        for j, augmentedDataList in enumerate(allAugmentedDataList):
            #Compute the similarities of the corresponding traduction
            semanticSimilarity = getSemanticSimilarity(originalText, augmentedDataList[i], model)
            lexicalSimilarity = getLexicalSimilarity(originalText, augmentedDataList[i])

            #Save the traduction and the similarities in a dictionary
            allAugmentedDataInfoDict.update({
                f"augmented{j + 1}": augmentedDataList[i],
                f"semanticSimilarity{j + 1}": semanticSimilarity,
                f"lexicalSimilarity{j + 1}": lexicalSimilarity
            })

            #Get the index of the traduction with greater semantic similarity and less lexical similarity
            bestIdx = max(bestIdx, j + 1,
                key = lambda k: (allAugmentedDataInfoDict[f"semanticSimilarity{k}"], -allAugmentedDataInfoDict[f"lexicalSimilarity{k}"])
            )

        #Select the information of the best augmentation
        info = {
            "originalText": originalText,
            "bestAugmentation": allAugmentedDataInfoDict[f"augmented{bestIdx}"],
            "bestAugmentedDataSemanticSimilarity": allAugmentedDataInfoDict[f"semanticSimilarity{bestIdx}"],
            "bestAugmentedDataLexicalSimiliratity": allAugmentedDataInfoDict[f"lexicalSimilarity{bestIdx}"]
        }
        info.update(allAugmentedDataInfoDict)

        #Save the information
        resul.append(info)

        #Write the information in the files
        withOriginalFile.write(originalText + "," + allAugmentedDataInfoDict[f"augmented{bestIdx}"] + "," + str(allAugmentedDataInfoDict[f"semanticSimilarity{bestIdx}"]) + "," + str(allAugmentedDataInfoDict[f"lexicalSimilarity{bestIdx}"]) + "\n")
        #If the text is not empty write it on  the file
        if allAugmentedDataInfoDict[f"augmented{bestIdx}"]  != '""':
            augmentedDataFile.write(allAugmentedDataInfoDict[f"augmented{bestIdx}"] + "\n")

    #Close the files
    withOriginalFile.close()
    augmentedDataFile.close()
    
    return resul

In [13]:
validWithOriginalPath = '1. Back Translation/3. Augmented Data/ValidBackTranslationWithOriginal.csv'
validAugmentedPath = '1. Back Translation/3. Augmented Data/ValidBackTranslationData.txt'
invalidWithOriginalPath = '1. Back Translation/3. Augmented Data/InvalidBackTranslationWithOriginal.csv'
invalidAugmentedPath = '1. Back Translation/3. Augmented Data/InvalidBackTranslationData.txt'

infoValid = processAugmentation(validOriginal, [validEsEnEnEsTraductionList, validEsJaJaEsTraductionList, validEsFrFrJaJaRuRuEsList], validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, [invalidEsEnEnEsTraductionList, invalidEsJaJaEsTraductionList, invalidEsFrFrJaJaRuRuEsList], invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [14]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...",“Tiene fácil acceso para personas con movilida...,0.978443,0.760000,“Tiene fácil acceso para personas con movilida...,0.978443,0.760000,“Es de fácil acceso para personas con movilida...,0.925599,0.566667,“Es de fácil acceso para personas con discapac...,0.854630,0.468750
1,"""Espero que hayan mejorais""","""Espero que hayas mejorado""",0.916635,0.333333,"""Espero que hayas mejorado""",0.916635,0.333333,"""Espero que las cosas estén mejorando"".",0.785516,0.250000,"""Espero que la situación esté mejorando"".",0.736162,0.250000
2,"""La estación es antigua, aparte de tener una s...",“La estación es antigua e inaccesible para per...,0.932935,0.500000,"“La estación es antigua, además de tener una ú...",0.883212,0.631579,“La estación es antigua e inaccesible para per...,0.932935,0.500000,“Además de que esta estación es antigua y tien...,0.932238,0.477273
3,"""Bien""","""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000,"""bien""",1.000000,1.000000,"""bien""",1.000000,1.000000
4,"""Bonito comodo""","""Bonito y cómodo""",0.955331,0.666667,"""Bonito y cómodo""",0.955331,0.666667,"""Maravilloso confort""",0.415461,0.000000,“Muy conveniente”,0.229466,0.000000


In [15]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...",“Llevo 35 años viviendo en el barrio y reconoz...,0.949282,0.631579,“Llevo 35 años viviendo en el barrio y reconoz...,0.949282,0.631579,"""He vivido en esta zona durante 35 años y reco...",0.818717,0.500000,“Vivo en esta zona desde hace 35 años y recono...,0.695063,0.418605
1,"""localización con muchos bares interesantes""","""Ubicación con muchos bares interesantes""",0.793883,0.666667,"""Ubicación con muchos bares interesantes""",0.793883,0.666667,"""Ubicación con muchos bares interesantes""",0.793883,0.666667,“Un lugar con muchos bares interesantes”,0.685196,0.571429
2,"""…""","""""",0.905728,0.000000,"""""",0.905728,0.000000,"""...""",0.838349,0.000000,"""""",0.905728,0.000000
3,"""Muy rica comida..""",“Comida muy sabrosa.”,0.706546,0.500000,"""Comida muy deliciosa..""",0.677902,0.500000,"""Es una comida muy deliciosa"".",0.644577,0.333333,“Comida muy sabrosa.”,0.706546,0.500000
4,"""Estación del.metro""","""Estación de metro""",0.963894,0.250000,"""Estación de metro""",0.963894,0.250000,"""estación de metro""",0.963894,0.250000,"""estación de metro""",0.963894,0.250000
